# Distracted Driving

## Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import matplotlib.pyplot as plt
import copy

Import Dataset, preprocess, add to loader_val

In [2]:
data_path = os.path.join('dset')
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))]) #Processing
train_dataset = dset.ImageFolder(root=data_path, transform=transform)
loader_val = torch.utils.data.DataLoader(train_dataset, batch_size=1)

Initialize VGG16

In [3]:
model = models.vgg16()

Freeze weights and modify architecture

In [4]:
for param in model.parameters():
    param.requires_grad = False

in_feat = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]
features.extend([nn.Linear(in_feat, 10)]) #Add layer with 10 class output
model.classifier = nn.Sequential(*features)

Retrieve weight

In [5]:
#Epoch 1 weights
weight0_path = os.path.join('model0.pth')
#Epoch 15 weights
weight_path = os.path.join('model.pth')

Function to load weights

In [6]:
def load_weights(model, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])

Make sure model is set for GPU

In [7]:
model = model.type(torch.cuda.FloatTensor)
model.cuda();

Helper function to calculate accuracy

In [8]:
# Modified from PyTorch.ipynb
def check_accuracy(model, loader):
    print('Checking Accuracy')
    num_correct = 0
    num_samples = 0
    model.eval()
    size = len(loader)
    percent = 0
    for n, (x,y) in enumerate(loader):
        if int(100*n/size) >= percent+10:
            percent = int(100*n/size)
            print(f'{percent}%: {n}/{size}')
        with torch.no_grad():
            x_var = x.cuda()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print(f'{num_correct}/{num_samples} correct - {100*acc}%')
    return 100*acc

Loads weights and checks accuracy

In [9]:
def check_model(model, path):
    load_weights(model, path)
    check_accuracy(model, loader_val)

Check accuracy of model with 1 epoch training

In [10]:
check_model(model, weight0_path)

Checking Accuracy
10%: 10/100
20%: 20/100
30%: 30/100
40%: 40/100
50%: 50/100
60%: 60/100
70%: 70/100
80%: 80/100
90%: 90/100
81/100 correct - 81.0%


Check accuracy of model with 15 epoch training

In [11]:
check_model(model, weight_path)

Checking Accuracy
10%: 10/100
20%: 20/100
30%: 30/100
40%: 40/100
50%: 50/100
60%: 60/100
70%: 70/100
80%: 80/100
90%: 90/100
95/100 correct - 95.0%
